In [1]:
import numpy as np

In [2]:
class Node:  # 把字理解为附着在边的，节点名字为数字
    def __init__(self,preNode : 'Node', allArrows: dict, nodeName : int):
        """
        创建一个字节点。

        一个节点应该考虑以下事情：
        1.从0节点到该节点的边的权值的和的最优值
        2.该节点有多少个相邻结点，就应该比较0节点通过这几个节点到达该节点的边的权值之和中，哪个是最低的。（小数的负对数，越小，越接近1）
        
        ----
        Attributes:
            self.name: int
                节点的名字
            self.preNode: <class, Node>
                前一个节点
            self.distanceFromZero: int
                0节点到该节点的最小距离
        ----
        Pamameter(s):
            preNode: <class, Node>
                该节点之前的可以直接到其的节点(最多只有一个)
            allArrows: dict of {arrowSerialNumber: <class, Arrow>}
                所有的有向边
            nodeName:
                当前节点的代号
        """


        """通过nodeName，然后遍历所有的边，取出所有目的地为该节点的边"""
        arrows = []  # 记录所有目的地为当前节点的边
        for arrow in allArrows.values():
            if arrow.endNode == nodeName:  # 该有向边的目的地是当前节点
                arrows.append(arrow)

        global allNodes

        self.name = nodeName
        if arrows == []:  # 该节点是第一个节点，也就是说要创建第一个节点
            self.distanceFromZero = 0
            self.preNode = None  #记录当前节点的前一个节点
            self.arrowFrom = None
        else:
            distances = []
            for arrow in arrows:
                arrowWeight = arrow.weight  # preNode经过arrow可以到达当前节点
                preNodeDistance = allNodes[ arrow.preNode ].distanceFromZero  # 每条边对应的preNode都不一样
                distances.append( preNodeDistance+arrowWeight )  # 记录0节点从每条路到当前节点的距离
            bestNodeIndex = np.argmin(distances)  # 选出最优距离
            self.distanceFromZero = distances[bestNodeIndex]
            self.preNode = allNodes[ arrows[bestNodeIndex].preNode ]  # bestNodeIndex对应arrow里面距离最小的，取出代号后得到Node
            self.arrowFrom = arrows[bestNodeIndex]


class Arrow:  
    def __init__(self,arrowName: str, arrowSerNum : int, preNode: int, endNode: int):
        """
        Attribute(s):
        self.name: str
            边的名字
        self.weight: float
            边从权重
        self.preNode: <class, Node>
            该边的出发节点
        ----
        Pramater(s):
            arrowName: 
                边的名字
            arrowSerNum:
                边的数字序列号
            preNode:
                该边的出发节点的数字代号
            endNode:
                该边到达的节点的数字代号
        """
        global ngram_logValue
        try:
            self.weight = ngram_logValue[arrowName]  # 该token是unigram
        except:
            self.weight = ngram_logValue[arrowName]  # 该token是bigram
        finally:
            self.name = arrowName
            self.serialNumber = arrowSerNum   
            self.preNode = preNode
            self.endNode = endNode


class Tokenizer:
    def __init__(self,file_ngram: 'file'):
        """
        中文分词
        ----
        Pamameter(s):
        file_ngram:
            ngram语料库
        """

        self.file_ngram = file_ngram
        self.ngramProbabilities = self.ngramProbability(self.file_ngram)  # 计算概率


    def encode(self,sentence : str) -> None:
        """
        最大化概率2-gram中文分词

        ----
        Parameter(s):
        sentence:
            输入
        """

        """
        1. 切割sentence,切成字的unigram和bigram（不是词的unigram和bigram）
        2. 计算每个gram出现的概率的负对数        
        """
        self.sentence = sentence

        unigram = [x for x in sentence]
        offsets = []
        for i in range(len(unigram)):
            offsets.append((i,i+len(unigram[i])))
        ngrams = self.ForwardMaxMatch(windowsLen=5) 
        i=0
        j=0
        while(i < len(sentence)):
            end = i+len(ngrams[j])
            offsets.append( (i,end ) )
            i = end 
            j += 1

        # ngrams = list(set(ngrams))  #去掉相同的gram
        ngrams = unigram+ngrams
        global ngram_logValue
        ngram_logValue = self.logProba_unigram(ngrams,self.ngramProbabilities)

        graph = self.CreateGraph(ngrams, offsets)  # 创建有向图
        
        return self.display(graph)


    def ForwardMaxMatch(self, windowsLen) -> list:
        """
        前向最大匹配算法
        
        ----
        Parameter(s):
            windowsLen:
                每次选取的字符串长度
        Return:

        """
        """
        从索引0开始，根据字符串长度选取一段子字符串，然后对这个子字符串做从
        右到左扫描，扫描过程中，末指针往前走，只要始末指针划定的字符串在词典中出现了，
        那么就把这个字符串作为一个ngram，然后把开始指针设定为末指针+1，继续扫描，
        直到初始指针等于句子的长度，表示初始指针已经走出字符串了，所以结束循环。
        """

        sentence = self.sentence
        start = 0
        result = []
        while start < len(sentence):
            end = min(len(sentence)-1, start+windowsLen-1)  # 防止越界
            for i in range(end,start-1,-1):
                subGram = sentence[start:i+1]
                if self.isInDict(subGram):
                    result.append(subGram)
                    start = i+1  #start跳到subGram的下一个字符那里
                    break  #匹配完成，进行下一次匹配
        return result
                

    def isInDict(self,subGram) -> bool:
        """判断某个gram是否在词典中"""
        return subGram in self.ngramProbabilities


    def display(self, graph: dict):
        Str = []
        end = len(graph)-1
        node = graph[end]
        while 1:
            try:
                Str.append( node.arrowFrom.name )
                node = node.preNode
            except:
                break
        return Str[::-1]


    def getNgramSpan(self, ngram: str) -> list:
        """
        得到ngram的出发节点和结束节点
        
        ----
        Return:
            [preNode,endNode]: ngram这条边的出发节点和结束节点
        """
        """
        ngram的始末相当于这个ngram在原始句子中的索引的始末，可以通过index方法
        得到ngram在句子中的初始位置，这个初始位置刚好对应出发节点（参见实验文档中的示意图），
        末位置在加个1，对应到达节点。
        """
        preNode = self.sentence.index(ngram)
        endNode = preNode+len(ngram)
        return [preNode,endNode]


    def CreateGraph(self, ngrams : list, offsets: list) -> dict:
        """
        根据unigram和其他ngram创建有向图
        ----
        Parameter(s):
            ngrams:
                所有的gram的集合
            offsets:
                这些gram当作边时的出发节点和到达节点
        Return:
            graph:
                有向图。
        """

        allArrows = {}
        # lenUnigram = len(unigram)
        # for i in range(lenUnigram):
        #     allArrows[i] = Arrow(unigram[i], i, i, i+1)  # i刚好对应边出发的节点的代号
        for i in range(len(ngrams)):
            preNode,endNode = offsets[i]
            allArrows[i] = Arrow(ngrams[i], i, preNode, endNode)

        global allNodes
        allNodes = {}
        allNodes[0] = Node(None,allArrows,0)
        allNodes[1] = Node(allNodes[0],allArrows,1)

        for i in range(2, len(self.sentence)+1):
            preNodes = allNodes[i-1]
            # allArrow： 0-lenUnigram-1是unigram的arrow，后面的则是bigram的arrow
            allNodes[i] = Node(preNodes,allArrows,i)
            
        graph = allNodes        
        return graph


    def logProba_unigram(self, ngrams : list ,ngramProbabilities: dict) -> dict:
        """
        计算ngrams中的每个gram在语料库中的统计概率

        ----
        Parameter(s):
            ngrams: 
                FMM产生的ngram的结果
            ngramProbabilities:
                语料库中所有ngram的概率的负对数
        Return:
            result:
                一个字典，例如{"我": 2.1231}
        """
        result = {}
        for gram in ngrams:
            try:
                probability = ngramProbabilities[gram]
            except:
                probability = self.getUnknowedProba()  # 语料库中没有这个字，那么使用加法平滑，给出一个小概率的负对数
            result[gram] = probability
        
        return result


    def logProba_bigram(self, bigram : list ,bigramProbabilities: dict) -> dict:
        """
        计算bigram中的每个字在语料库中的统计概率

        ----
        Parameter(s):
            bigram: 
                单个字的gram
            bigramProbabilities:
                语料库中所有unigram的概率的负对数
        Return:
            result:
                一个字典，例如{"我": 2.1231}
        """
        result = {}
        for gram in bigram:
            try:
                probability = bigramProbabilities[gram]
            except:
                probability = self.getUnknowedProba_bigram()  # 语料库中没有这个字，那么使用加法平滑，给出一个小概率的负对数
            result[gram] = probability
        
        return result


    def getUnknowedProba(self) -> int:
        """
        使用加法平滑，给出一个未知词的概率的负对数

        ----
        Return:
            proLog:
                概率的负对数
        """
        probability = 1/(len(self.ngramProbabilities) + len(self.ngramProbabilities))
        return -np.log(probability)


    def getUnknowedProba_bigram(self) -> int:
        """
        使用加法平滑，给出一个未知词的概率的负对数

        ----
        Return:
            proLog:
                概率的负对数
        """
        probability = 1/(len(self.bigramProbabilities) + len(self.bigramProbabilities))
        return -np.log(probability)


    def ngramProbability(self,file_ngram) -> dict:
        """
        计算unigram的-log概率
        ----
        Pamater(s):
            file_ngram: file
                ngram语料库
        Return:
            myDict: dict
                每个unigram对应的-log对数
        """
        with open(file_ngram, encoding='utf-8') as F:
            myDict = {}
            allLines = F.readlines()
            total_words = 0
            for line in allLines:
                line = line.split()
                total_words += int(line[1])

            for line in allLines:
                line = line.split()
                myDict[line[0]] = -np.log(int(line[1])/total_words)

        return myDict


    def bigramProbability(self,file_bigram ) -> dict:
        """
        计算bigram的-log概率
        ----
        Pamater(s):
            file_bigram: file
                bigram语料库
        Return:
            myDict: dict
                每个bigram对应的-log对数
        """   
        with open(file_bigram, 'r') as F:
            myDict = {}
            allLines = F.readlines()
            total_words = 0
            for line in allLines:
                line = line.split()
                total_words += int(line[1])

            for line in allLines:
                line = line.split()
                myDict[line[0]] = -np.log(int(line[1])/total_words)   

        return myDict

In [50]:

    with open("人民日报96年语料.124-gram.word", 'x', encoding='utf-8') as newF:
        for i in [1,2,4]:
            print(i)
            with open("人民日报96年语料.{}-gram.word".format(i), encoding='utf-8') as F:
                for line in F:
                    if line != None:
                        newF.write(line)


1
2
4


In [3]:
sentence = "我是顾小杰我今年二十岁我在巴基斯坦卖包子"
file_ngram = "人民日报96年语料.124-gram.word"
Tokenizer(file_ngram).encode(sentence)

['我是', '顾小', '杰', '我', '今', '年二', '十岁', '我在', '巴基斯坦', '卖', '包子']

In [54]:
sentence = "我是顾小杰我在巴基斯坦卖包子"
file_ngram = "人民日报96年语料.unigram"
Tokenizer(file_ngram).encode(sentence)

['我', '是', '顾', '小', '杰', '我', '在', '巴基斯坦', '卖', '包子']

In [43]:
class Node:  # 把字理解为附着在边的，节点名字为数字
    def __init__(self,preNode : 'Node', arrows: list, nodeName : int):
        """
        创建一个字节点。

        一个节点应该考虑以下事情：
        1.从0节点到该节点的边的权值的和的最优值
        2.该节点有多少个相邻结点，就应该比较0节点通过这几个节点到达该节点的边的权值之和中，哪个是最低的。（小数的负对数，越小，越接近1）
        
        ----
        Attributes:
            self.name: int
                节点的名字
            self.preNode: <class, Node>
                前一个节点
            self.distanceFromZero: int
                0节点到该节点的最小距离
        ----
        Pamameter(s):
            preNode: <class, Node>
                该节点之前的可以直接到其的节点(最多只有一个)
            arrows: list of <class, Arrow>
                里面的元素与preNode的元素一一对应，表示preNode通过arrow可以到达当前Node
            nodeName:
                当前节点的代号
        """

        global allNodes

        self.name = nodeName
        if preNode == None:  # 该节点是第一个节点，也就是说要创建第一个节点
            self.distanceFromZero = 0
            self.preNode = None  #记录当前节点的前一个节点
            self.arrowFrom = None
        else:
            distances = []
            for arrow in arrows:
                arrowWeight = arrow.weight  # preNode经过arrow可以到达当前节点
                preNodeDistance = allNodes[ arrow.preNode ].distanceFromZero  # 每条边对应的preNode都不一样
                distances.append( preNodeDistance+arrowWeight )  # 记录0节点从每条路到当前节点的距离
            bestNodeIndex = np.argmin(distances)  # 选出最优距离
            self.distanceFromZero = distances[bestNodeIndex]
            self.preNode = allNodes[ arrows[bestNodeIndex].preNode ]  # bestNodeIndex对应arrow里面距离最小的，取出代号后得到Node
            self.arrowFrom = arrows[bestNodeIndex]


class Arrow:  
    def __init__(self,arrowName: str, arrowSerNum : int, preNode: int):
        """
        Attribute(s):
        self.name: str
            边的名字
        self.weight: float
            边从权重
        self.preNode: <class, Node>
            该边的出发节点
        ----
        Pramater(s):
            arrowName: 
                边的名字
            arrowSerNum:
                边的数字序列号
            preNode:
                该边的出发节点的数字代号
        """
        global unigram_logValue,bigram_logValue
        try:
            self.weight = unigram_logValue[arrowName]  # 该token是unigram
        except:
            self.weight = bigram_logValue[arrowName]  # 该token是bigram
        finally:
            self.name = arrowName
            self.serialNumber = arrowSerNum   
            self.preNode = preNode


class Tokenizer:
    def __init__(self,file_unigram: 'file', file_bigram : 'file'):
        """
        中文分词
        ----
        Pamameter(s):
        file_unigram:
            unigram语料库
        file_bigram:
            bigram语料库
        """

        self.file_unigram = file_unigram
        self.file_bigram = file_bigram
        
        self.unigramProbabilities = self.unigramProbability(self.file_unigram)  # 计算概率
        self.bigramProbabilities = self.bigramProbability(self.file_bigram)  # 计算概率


    def encode(self,sentence : str) -> None:
        """
        最大化概率2-gram中文分词

        ----
        Parameter(s):
        sentence:
            输入
        """

        """
        1. 切割sentence,切成字的unigram和bigram（不是词的unigram和bigram）
        2. 计算每个gram出现的概率的负对数        
        """
        unigram = [x for x in sentence]
        bigram = [sentence[i:i+2] for i in range(len(sentence)-1)]

        global unigram_logValue,bigram_logValue
        unigram_logValue = self.logProba_unigram(unigram,self.unigramProbabilities)
        bigram_logValue = self.logProba_bigram(bigram,self.bigramProbabilities)

        graph = self.CreateGraph(unigram, bigram)  # 创建有向图
        
        return self.display(graph)


    def display(self, graph: dict):
        Str = []
        end = len(graph)-1
        node = graph[end]
        while 1:
            try:
                Str.append( node.arrowFrom.name )
                node = node.preNode
            except:
                break
        return Str[::-1]


    def CreateGraph(self, unigram : list, bigram : list) -> dict:
        """
        根据unigram和bigram创建有向图
        ----
        Return:
            graph:
                有向图。
        """

        allArrows = {}
        lenUnigram = len(unigram)
        for i in range(lenUnigram):
            allArrows[i] = Arrow(unigram[i], i, i)  # i刚好对应边出发的节点的代号
        for i in range(len(bigram)):
            allArrows[i+lenUnigram] = Arrow(bigram[i], i+lenUnigram, i)

        global allNodes
        allNodes = {}
        allNodes[0] = Node(None,[],0)
        allNodes[1] = Node(allNodes[0],[allArrows[0]],1)

        for i in range(2, len(unigram)+1):
            preNodes = allNodes[i-1]
            arrows = [ allArrows[ i-1 ], allArrows[ i-2+lenUnigram ] ]  # 0后面的节点都有两条边考虑
            # allArrow： 0-lenUnigram-1是unigram的arrow，后面的则是bigram的arrow
            allNodes[i] = Node(preNodes,arrows,i)
            
        graph = allNodes        
        return graph


    def logProba_unigram(self, unigram : list ,unigramProbabilities: dict) -> dict:
        """
        计算unigram中的每个字在语料库中的统计概率

        ----
        Parameter(s):
            unigram: 
                单个字的gram
            unigramProbabilities:
                语料库中所有unigram的概率的负对数
        Return:
            result:
                一个字典，例如{"我": 2.1231}
        """
        result = {}
        for gram in unigram:
            try:
                probability = unigramProbabilities[gram]
            except:
                probability = self.getUnknowedProba_unigram()  # 语料库中没有这个字，那么使用加法平滑，给出一个小概率的负对数
            result[gram] = probability
        
        return result


    def logProba_bigram(self, bigram : list ,bigramProbabilities: dict) -> dict:
        """
        计算bigram中的每个字在语料库中的统计概率

        ----
        Parameter(s):
            bigram: 
                单个字的gram
            bigramProbabilities:
                语料库中所有unigram的概率的负对数
        Return:
            result:
                一个字典，例如{"我": 2.1231}
        """
        result = {}
        for gram in bigram:
            try:
                probability = bigramProbabilities[gram]
            except:
                probability = self.getUnknowedProba_bigram()  # 语料库中没有这个字，那么使用加法平滑，给出一个小概率的负对数
            result[gram] = probability
        
        return result


    def getUnknowedProba_unigram(self) -> int:
        """
        使用加法平滑，给出一个未知词的概率的负对数

        ----
        Return:
            proLog:
                概率的负对数
        """
        probability = 1/(len(self.unigramProbabilities) + len(self.unigramProbabilities))
        return -np.log(probability)


    def getUnknowedProba_bigram(self) -> int:
        """
        使用加法平滑，给出一个未知词的概率的负对数

        ----
        Return:
            proLog:
                概率的负对数
        """
        probability = 1/(len(self.bigramProbabilities) + len(self.bigramProbabilities))
        return -np.log(probability)


    def unigramProbability(self,file_unigram) -> dict:
        """
        计算unigram的-log概率
        ----
        Pamater(s):
            file_unigram: file
                unigram语料库
        Return:
            myDict: dict
                每个unigram对应的-log对数
        """
        with open(file_unigram, encoding='utf-8') as F:
            myDict = {}
            allLines = F.readlines()
            total_words = 0
            for line in allLines:
                line = line.split()
                total_words += int(line[1])

            for line in allLines:
                line = line.split()
                myDict[line[0]] = -np.log(int(line[1])/total_words)

        return myDict


    def bigramProbability(self,file_bigram ) -> dict:
        """
        计算bigram的-log概率
        ----
        Pamater(s):
            file_bigram: file
                bigram语料库
        Return:
            myDict: dict
                每个bigram对应的-log对数
        """   
        with open(file_bigram, encoding='utf-8') as F:
            myDict = {}
            allLines = F.readlines()
            total_words = 0
            for line in allLines:
                line = line.split()
                total_words += int(line[1])

            for line in allLines:
                line = line.split()
                myDict[line[0]] = -np.log(int(line[1])/total_words)   

        return myDict

In [44]:
sentence = "我在巴基斯坦卖包子"
file_unigram = "人民日报96年语料.unigram.word"
file_bigram = "人民日报96年语料.bigram.word"
Tokenizer(file_unigram,file_bigram).encode(sentence)

['我', '在', '巴基', '斯坦', '卖包', '子']